In [1]:
import keras
import sys

import numpy as np
import pandas as pd 
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Input, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import backend as K
from keras.regularizers import l2

Using TensorFlow backend.
/home/linnik/.conda/envs/linnik/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/linnik/.conda/envs/linnik/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/linnik/.conda/envs/linnik/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/linnik/.conda/envs/linn

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
train_data_dir = "/home/linnik/My_dataset_new/train"   
validation_data_dir = "/home/linnik/My_dataset_new/val"

In [4]:
img_width, img_height = 70, 70
nb_train_samples = 8403
nb_validation_samples = 502
epochs = 50
batch_size = 8
n_classes = 11

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    zoom_range=0.2,
    rotation_range = 5,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

Found 8403 images belonging to 11 classes.
Found 502 images belonging to 11 classes.


In [7]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [8]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [9]:
def resNetM(input_shape, classes):
    img_input = Input(shape=input_shape)

    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = AveragePooling2D((3, 3), name='avg_pool')(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu', name='fc01')(x)
    x = Dense(200, activation='relu', name='fc02')(x)
    x = Dropout(0.3, name='dr01')(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)

    inputs = img_input

    model = Model(inputs, x, name='resnetM')

    return model

In [10]:
model = resNetM((img_width, img_height, 3), n_classes)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [11]:
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)

Model: "resnetM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70, 70, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 35, 35, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 35, 35, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 35, 35, 64)   0           bn_conv1[0][0]                   
____________________________________________________________________________________________

In [12]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
check = ModelCheckpoint("Best_ResNetM_50_epochs.hdf5", monitor = "val_acc", save_best_only = True) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr, check]

In [17]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
1050/1050 [==============================] - 45s 43ms/step - loss: 3.3378 - acc: 0.2633 - val_loss: 2.9627 - val_acc: 0.2560
Epoch 2/50
1050/1050 [==============================] - 30s 29ms/step - loss: 1.2781 - acc: 0.5350 - val_loss: 3.2056 - val_acc: 0.4433
Epoch 3/50
1050/1050 [==============================] - 33s 32ms/step - loss: 0.8638 - acc: 0.6584 - val_loss: 3.4255 - val_acc: 0.4049
Epoch 4/50
1050/1050 [==============================] - 36s 34ms/step - loss: 0.6254 - acc: 0.7492 - val_loss: 2.9622 - val_acc: 0.6559
Epoch 5/50
1050/1050 [==============================] - 31s 30ms/step - loss: 0.4424 - acc: 0.8254 - val_loss: 3.2120 - val_acc: 0.5607
Epoch 6/50
1050/1050 [==============================] - 38s 36ms/step - loss: 0.3365 - acc: 0.8710 - val_loss: 2.9516 - val_acc: 0.6012
Epoch 7/50
1050/1050 [==============================] - 31s 29ms/step - loss: 0.2752 - acc

In [18]:
loaded_model = load_model("Best_ResNetM_50_epochs.hdf5")

In [19]:
right = 0
wrong = 0
times = []

for folder in os.listdir(validation_data_dir):
    for img in os.listdir(validation_data_dir + "/" + folder):
        imag = im.load_img(validation_data_dir + "/" + folder + "/" + img, target_size = (img_width, img_height))
        
        photo_ar = im.img_to_array(imag)
        photo_ar = np.expand_dims(photo_ar, axis = 0)
        photo_ar /= 255
        
        start_time = time.time()
        ans = np.argmax(loaded_model.predict(photo_ar))
        times.append(time.time() - start_time)
        
        if ans == int(folder)-1:
            right += 1
        else:
            wrong += 1

In [20]:
print("Accuracy on validation set is", right/(right + wrong))
print("Average time to pridict one photo is", np.array(times).mean(), "seconds")

Accuracy on validation set is 0.7430278884462151
Average time to pridict one photo is 0.008097670467726263 seconds


In [21]:
# Проверка качества сети по классам (Precision and Recall)

TP = [0 for i in range(n_classes)]
FN = [0 for i in range(n_classes)]
FP = [0 for i in range(n_classes)]

precision = [0 for i in range(n_classes)]
recall = [0 for i in range(n_classes)]

for folder in os.listdir(validation_data_dir):
    for img in os.listdir(validation_data_dir + "/" + folder):
        imag = im.load_img(validation_data_dir + "/" + folder + "/" + img, target_size = (img_width, img_height))
        
        photo_ar = im.img_to_array(imag)
        photo_ar = np.expand_dims(photo_ar, axis = 0)
        photo_ar /= 255
        
        ans = np.argmax(loaded_model.predict(photo_ar))
    
        if ans == int(folder)-1:
            TP[ans] += 1
        if ans != int(folder)-1:
            FN[ans] += 1
            FP[int(folder)-1] += 1

In [22]:
for i in range(n_classes):
    try:
        precision[i] = TP[i]/(TP[i] + FP[i])
    except:
        precision[i] = 0
        
    try:
        recall[i] = TP[i]/(TP[i] + FN[i])
    except:
        recall[i] = 0
    
precision = np.array(precision)
recall = np.array(recall)

print(precision.max(), precision.min(), precision.mean())
print(recall.max(), recall.min(), recall.mean())

1.0 0.0 0.5609900851836336
0.9021739130434783 0.0 0.45350341358246504
